<h1 style="text-align:center;">Predicting heart disease</h1>

In [12]:
import pandas as pd
import numpy as np
import warnings

import matplotlib.pyplot as plt

# Import train_test_split
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV

# Import Decision Tree classifier
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor

# Import accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.base import BaseEstimator, TransformerMixin

from sklearn import tree
from sklearn.metrics import mean_squared_error, accuracy_score

from category_encoders import OneHotEncoder

from helper_file import *

warnings.filterwarnings('ignore')

In [4]:
heart_url = "data/heart_disease.csv"

In [6]:
df_heart = (pd
 .read_csv(heart_url)
)

df_heart.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


Alright, let's have a little chat about what each of these terms you've got here means. It's really not all that complicated, I promise!

1. **`age`**: This is just the number of years someone has been around - simple enough, right?

2. **`sex`**: Here we're identifying whether someone is male (1) or female (0). A binary kind of thing!

3. **`cp`** or **Chest Pain Type**: Now, here we're categorizing the kind of chest pain someone might be experiencing. It could be "typical angina" which is kind of the classic type of chest pain, "atypical angina" which is a bit different, "non-anginal pain" which isn't related to the heart, or "asymptomatic" which means the person doesn't feel any pain at all.

4. **`trestbps`** or **Resting Blood Pressure**: This is the pressure in someone's arteries when they're not doing anything strenuous - just chilling out, you might say. It's measured in mm Hg.

5. **`chol`** or **Cholesterol**: This one is the amount of cholesterol floating around in someone's blood, measured in mg/dl.

6. **`fbs`** or **Fasting Blood Sugar**: Here we're checking if the sugar level in the blood is higher than 120 mg/dl after not eating for a while (fasting). It's a yes (1) or no (0) kind of deal.

7. **`restecg`** or **Resting Electrocardiographic Results**: This is a fancy way of talking about the results of a heart test. It might show everything is normal (0), some small irregularities (1), or signs that the left part of the heart is working too hard (2).

8. **`thalach`** or **Maximum Heart Rate Achieved**: This is the fastest the heart was beating during the test - a peak performance, if you will!

9. **`exang`** or **Exercise Induced Angina**: Here we're noting whether exercise causes chest pain. Again, it's a yes (1) or no (0) kind of situation.

10. **`oldpeak`**: This is a measure of a specific change in the heart test when someone is exercising compared to when they are resting.

11. **`slope`**: This is looking at another part of the heart test - specifically, the shape of a certain curve when someone is working out. It might go up (1), stay flat (2), or go down (3).

12. **`ca`** or **Number of Major Vessels**: Here, we're counting the number of big blood vessels that are visible in a certain type of test - it can be anywhere from 0 to 3.

13. **`thal`**: This is a test that looks at the heart's blood supply. It might be normal (3), have a permanent defect (6), or have a defect that can change (7).

Now, to make use of this data for learning, we'll need to split it into two groups: one to help our machine learn, and another to test how well it has learned. It's just like giving a student a set of practice problems and then a separate set of exam problems to see how well they've understood the material!

In [7]:
X, y = splitX_y(df_heart, 'target')

print(f"shape of target vector: {y.shape}")
print(f"shape of feature matrix: {X.shape}")

shape of target vector: (303,)
shape of feature matrix: (303, 13)


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=43)

In [9]:
model = DecisionTreeClassifier(random_state=43)

scores = cross_val_score(model, X, y, cv=5)

print(f'Accuracy: {np.round(scores, 2)}')

print(f'Accuracy mean: {scores.mean():.2f}' )

Accuracy: [0.74 0.84 0.74 0.75 0.72]
Accuracy mean: 0.76


You see, sometimes, when we are fine-tuning our machine learning models, we got a heck of a lot of settings to adjust, which we call "hyperparameters". Now, tweaking these one by one using `GridSearchCV` can feel like waiting for grass to grow - it's a slow process, my friend.

But fear not, because the folks who crafted the `scikit-learn` library threw us a lifeline here with something they call `RandomizedSearchCV`. Think of it as a fun game of lottery where, instead of meticulously trying every single combination of numbers, you try your luck with a few random combinations hoping to hit the jackpot. It's a clever trick to save time!

So, instead of sifting through every possible combination, which might take ages, this method plays around with a random selection of these combinations, giving you a fair shot at finding the golden settings without spending an eternity on it. It's like a clever shortcut to finding some of the best combinations without having to turn over every stone in the field. It's not about being thorough, it's about being smart and efficient with the time and resources we've got.

In [11]:
def randomized_search_clf(df, trgt_vect, params, model, runs=20):
    """
    Ah, hello there! Welcome to the exciting journey of optimizing a classifier!
    
    This function takes in a data frame and a bunch of other parameters and does a little dance with 
    RandomizedSearchCV to find the best model settings. You see, instead of testing every single 
    combination of hyperparameters (which can be a bit like watching paint dry), we're going to 
    smartly sample a few combinations randomly. It's a quicker way to get us a good-enough model without 
    spending an eternity on it.
    
    Parameters:
    df (DataFrame): The data frame containing your data.
    trgt_vect (Series): The target variable you're trying to predict.
    params (dict): The range of hyperparameters you want to test out.
    runs (int, optional): The number of random combinations to try. Defaults to 20.
    model (estimator object): The base model you want to optimize.
    
    Returns:
    best_model (estimator object): The optimized model that performed the best in our little experiment.
    
    Now, let's roll up our sleeves and let the computer do the heavy lifting!
    """
    
    X, y = splitX_y(df_heart, trgt_vect)

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=43)
    
    rand_clf = RandomizedSearchCV(model, params, n_iter=runs, cv=5, n_jobs=-1, random_state=2)
    rand_clf.fit(X_train, y_train)

    best_model = rand_clf.best_estimator_
    best_score = rand_clf.best_score_

    print(f"Training score: {best_score:.3f}")

    y_pred = best_model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)

    print(f'Test score: {accuracy:.3f}')

    return best_model

When it comes to selecting these things called "hyperparameters", there's no straight arrow, no one-size-fits-all solution. You see, it's a bit like jazz improvisation; there's a certain freedom, a certain joy in trying out different tunes to see what works best. In our script, nestled within the `randomized_search_clf` function, we've laid out an initial selection of numbers, a starting point if you will. These numbers aren't just plucked out of thin air; they're chosen carefully to cast a wide net and to keep our journey steady, avoiding too many bumps and jolts.

In [13]:
params={'criterion':['entropy', 'gini'],'splitter':['random', 'best'], 
        'min_weight_fraction_leaf':[0.0, 0.0025, 0.005, 0.0075, 0.01],
        'min_samples_split':[2, 3, 4, 5, 6, 8, 10],
        'min_samples_leaf':[1, 0.01, 0.02, 0.03, 0.04],
        'min_impurity_decrease':[0.0, 0.0005, 0.005, 0.05, 0.10, 0.15, 0.2],
        'max_leaf_nodes':[10, 15, 20, 25, 30, 35, 40, 45, 50, None],
        'max_features':['auto', 0.95, 0.90, 0.85, 0.80, 0.75, 0.70],
        'max_depth':[None, 2,4,6,8],
        'min_weight_fraction_leaf':[0.0, 0.0025, 0.005, 0.0075, 0.01, 0.05]}

In [14]:
randomized_search_clf(df=df_heart, trgt_vect='target', 
                      params=params, model=model, 
                      runs=20)

Training score: 0.753
Test score: 0.816


DecisionTreeClassifier(criterion='entropy', max_depth=4, max_features=0.75,
                       max_leaf_nodes=25, min_impurity_decrease=0.0005,
                       min_samples_split=5, min_weight_fraction_leaf=0.0075,
                       random_state=43)

Alright, let's have a little heart-to-heart about fine-tuning our machine learning model here. You see, one trick up our sleeve is to tighten the reins on the range of hyperparameters we're exploring.

For instance, if we've noticed that a `max_depth` of 8 is doing wonders in our best model, we might just decide to play around in the neighborhood, maybe peek at what happens at 7 or 9.

Now, here's another nugget of wisdom - if a default setting is holding its ground pretty well, why bother messing with it, right? Like, in our case, `entropy` doesn't really bring a lot to the table compared to `gini`, it's a marginal difference really. So, we might as well stick with `gini` and let `entropy` sit this one out. The same goes for `min_impurity_split` and `min_impurity_decrease`; if they're doing fine as they are, let's not lose sleep over them.

Moving on, I've brewed a fresh batch of hyperparameter ranges for us to try, and I've pumped up the runs to a solid 100 to give us a better shot at finding that sweet spot. Here, take a look at this configuration for our randomized_search_clf:

In [15]:
params={'max_depth':[None, 6, 7],'max_features':['auto', 0.78], 
        'max_leaf_nodes':[45, None], 'min_samples_leaf':[1, 0.035, 0.04, 0.045, 0.05],
        'min_samples_split':[2, 9, 10],'min_weight_fraction_leaf': [0.0, 0.05, 0.06, 0.07],}

In [16]:
randomized_search_clf(df=df_heart, trgt_vect='target', 
                      params=params, model=model, 
                      runs=100)

Training score: 0.793
Test score: 0.763


DecisionTreeClassifier(max_depth=6, max_features='auto', min_samples_leaf=0.035,
                       random_state=43)

In [19]:
model = DecisionTreeClassifier(class_weight=None, 
                               criterion='gini', max_depth=7,
                               max_features='auto', max_leaf_nodes=45, 
                               min_impurity_decrease=0.0, 
                               min_samples_leaf=0.035, 
                               min_samples_split=9, min_weight_fraction_leaf=0.06, 
                               random_state=43, splitter='best')
scores = cross_val_score(model, X, y, cv=5)

print(f'Accuracy:{np.round(scores, 2)}')

print(f'Accuracy mean:{(scores.mean()):.2f}')

Accuracy:[0.75 0.8  0.74 0.75 0.77]
Accuracy mean:0.76


### `feature_importances_`: Unveiling the Key Players

Alright, let's cut to the chase. We're at a stage where we want to spotlight the shining stars, the key features that our machine learning model deems most important. Thankfully, decision trees hand us a neat tool for this, termed `feature_importances_`.

Now, before we dive in, we've got to settle on our champion model. You see, our function has pinpointed the best contender, but we haven't officially crowned it yet; it's not saved.

Here's a little nugget of wisdom for the testing phase: keep your training and test sets separate to avoid any mix-ups. But once we've zeroed in on our final model, it's a good strategy to let it loose on the entire dataset. This move might just give our model a little extra edge, a boost in accuracy, as it gets to learn from a richer set of data, prepping it to face new, unseen data with confidence.

So, let's roll up our sleeves and get our best model set up with the finest hyperparameters, ready to be trained on the full dataset:

In [21]:
best_clf = DecisionTreeClassifier(class_weight=None, 
                                  criterion='gini', 
                                  max_depth=9,max_features=0.8, 
                                  max_leaf_nodes=47,min_impurity_decrease=0.0, 
                                  min_samples_leaf=1, 
                                  min_samples_split=8, min_weight_fraction_leaf=0.05, 
                                  random_state=43, splitter='best')

best_clf.fit(X, y)

DecisionTreeClassifier(max_depth=9, max_features=0.8, max_leaf_nodes=47,
                       min_samples_split=8, min_weight_fraction_leaf=0.05,
                       random_state=43)

In [22]:
best_clf.feature_importances_

array([0.03458176, 0.        , 0.24669228, 0.        , 0.02326202,
       0.        , 0.01439797, 0.06288718, 0.        , 0.0390193 ,
       0.        , 0.17279501, 0.40636449])

In [23]:
# Zip columns and feature_importances_ into dict
feature_dict = dict(zip(X.columns, best_clf.feature_importances_))

# Import operator
import operator

# Sort dict by values (as list of tuples)
sorted(feature_dict.items(), key=operator.itemgetter(1), reverse=True)[0:3]

[('thal', 0.4063644861324159),
 ('cp', 0.24669227792723478),
 ('ca', 0.17279500568541162)]

Alright, my fellow explorers, let's gather 'round as we unveil the three superstar features that are taking the lead in our model:

1. **'cp'**: This denotes the type of chest pain experienced. It's categorized as follows:
   - **1**: Typical Angina
   - **2**: Atypical Angina
   - **3**: Non-Anginal Pain
   - **4**: Asymptomatic

2. **'thalach'**: This represents the pinnacle of the heart rate that has been achieved.
   
3. **'ca'**: This indicates the count of major vessels that are highlighted under fluoroscopy, ranging from 0 to 3.

Now, let's delve a bit deeper. You see, these numbers aren't just arbitrary; they have a story to tell. The 'cp' or chest pain type, for instance, is a heavyweight, accounting for a whopping 48% of the variance, outshining 'thal' and 'ca' combined.

Now, here's a message we can share with our medical team: Our model stands as a reliable ally, predicting the presence of heart disease with an impressive accuracy of 82%. And it does so by focusing its lens on three vital signs: the nature of the chest pain, the peak heart rate achieved, and the fluoroscopy results of the major vessels. It's a collaborative effort where chest pain, heart rate, and fluoroscopy results come together to paint a comprehensive picture.